# Cube

$\sigma=0.1l_e$

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'cube1'
r = SampleUnion(proj_name)

canvas = [["noise", "BF", "BNF", "BGF"],
          ["vert", "edge", "area", "area_rf"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 1)
show_metrics = quick_gen_vis_type(canvas, 1)
# print(vis_type)

# exp is used for my thesis
r.show(canvas, fig_name='exp/{}.png'.format(proj_name), 
       mesh_vis_type=vis_type, 
       show_metrics=show_metrics,
       titles=canvas)

$\sigma=0.5l_e$

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'cube5'
r = SampleUnion(proj_name)

canvas = [["noise", "BF", "BNF", "BGF"],
          ["vert", "edge", "area", "area_rf"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 1)
show_metrics = quick_gen_vis_type(canvas, 1)
# print(vis_type)

r.show(canvas, fig_name='exp/{}.png'.format(proj_name), 
       mesh_vis_type=vis_type, 
       show_metrics=show_metrics,
       titles=canvas)

$\sigma=0.9l_e$

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'cube9'
r = SampleUnion(proj_name)

canvas = [["noise", "BF", "BNF", "BGF"],
          ["vert", "edge", "area", "area_rf"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 1)
show_metrics = quick_gen_vis_type(canvas, 1)
# print(vis_type)

r.show(canvas, fig_name='exp/{}.png'.format(proj_name), 
       mesh_vis_type=vis_type, 
       show_metrics=show_metrics,
       titles=canvas)

# Selected Figures

## Synthetic

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'selected'
r = SampleUnion(proj_name)
"""
for j in r.results:
    print(j)
"""

canvas = [["part_Lp", "trim-star", "fandisk", "octa-flower", "block"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])
# print(vis_type)

r.show(canvas, fig_name='exp/{}-syn1.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics)

In [ ]:
canvas = [["fertility", "joint", "bunny", "chinese_lion", "dragon"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])
# print(vis_type)

r.show(canvas, fig_name='exp/{}-syn2.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics)

## Kinect v1

In [ ]:
canvas = [["big_girl_01", "cone_01", "david_01", "pyramid_01", "boy_01"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])

r.show(canvas, fig_name='exp/{}-kv1.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics, camera_position='xy')

## Kinect v2

In [ ]:
canvas = [["big_girl_03", "cone_03", "david_03", "pyramid_03", "boy_03"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])
# print(vis_type)

r.show(canvas, fig_name='exp/{}-kv2.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics, camera_position='xy')

## Kinect Fusion

In [ ]:
canvas = [["big_girl", "cone", "girl", "boy02", "david"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])
# print(vis_type)

r.show(canvas, fig_name='exp/{}-kf.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics, camera_position='xy')

# 非规整网格

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'ir1'
r = SampleUnion(proj_name)


canvas = [["40359", "40359edge", "40359area", "40359area_r"],
             ["40447", "40447edge", "40447area", "40447area_r"],
             ["40843", "40843edge", "40843area", "40843area_r"],
             ["43960", "43960edge", "43960area", "43960area_r"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 0)
show_metrics = quick_gen_vis_type(canvas, 1)

r.show(canvas, fig_name='exp/{}.png'.format(proj_name), mesh_vis_type=vis_type, show_metrics=show_metrics)


# Robust

通过以下脚本简单制作`test_robust.bat`中的加权参数（如果需要修改的话运行）

为了美观，考虑使用以下配色 https://zhuanlan.zhihu.com/p/601170660 


In [ ]:
"""
for i in range(1,41):
    print('set m{}={:.1f}'.format(i,i*0.1))
"""

color1 = (20/255,54/255,95/255) 
color2 = (214/255,79/255,56/255)

color3 = (118/255,162/255,185/255)
color4 = (191/255,217/255,229/255)
color5 = (248/255,242/255,236/255)

## Block ($\sigma=0.2l_e$)

对CAD模型的测试（一般来说CAD模型的超参数和非CAD模型的超参数不一致）

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scripts.visualize import *

def gen_robust_figure(proj_name, xlim=(0.6, 1.5), ylim=(2.55, 2.75),
                      loc=(0.2, 0.4, 0.4, 0.4),
                      bbox_color = (191/255,217/255,229/255)):
    r = SampleUnion(proj_name)
    AADs = []
    AHDs = []
    for k in r.results:
        AADs.append(r.results[k].AAD)
        AHDs.append(r.results[k].AHD*1000)
    

    # visual settings
    font = {'family' : 'Times New Roman',
            'size'   : 12}
    plt.rc('font', **font)

    # ---------------------------------------------------------------------------------
    # https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html #
    # ---------------------------------------------------------------------------------

    # Create some mock data
    t = np.array([0.1*i for i in range(1, len(AADs)+1)])
    data1 = np.array(AADs)
    data1_ = 'Average Angular Distance'
    data2 = np.array(AHDs)
    data2_ = 'Average Hausdorff Distance (10-3)'

    fig, ax1 = plt.subplots()

    color1 = (20/255,54/255,95/255)
    ax1.set_xlabel("multiplicator", fontsize=12)
    ax1.set_ylabel(data1_, color=color1)
    ax1.plot(t, data1, color=color1)
    ax1.tick_params(axis='y', labelcolor='black')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color2 = (214/255,79/255,56/255)
    ax2.set_ylabel(data2_, color=color2)  # we already handled the x-label with ax1
    ax2.plot(t, data2, color=color2)
    ax2.tick_params(axis='y', labelcolor='black')

    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    #-----------------------------------------
    # https://zhuanlan.zhihu.com/p/136574534 #
    #-----------------------------------------

    axins = ax1.inset_axes(loc)
    axins.plot(t, data1, color=color1)


    # 调整子坐标系的显示范围
    axins.set_xlim(xlim[0], xlim[1])
    axins.set_ylim(ylim[0], ylim[1])

    mark_inset(ax1, axins, loc1=3, loc2=4, fc="none", ec=bbox_color, lw=1)
    plt.savefig('exp/{}.png'.format(proj_name), dpi=300, bbox_inches='tight')
    plt.show()

gen_robust_figure('robust_block')


## nicolo

In [ ]:
gen_robust_figure('robust_nicolo', xlim=(0.3,0.9), ylim=(6.4,7), 
                  loc=(0.5, 0.2, 0.4, 0.3))

## 不同编译选项对速度的影响

绮良良的颜色

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c1,c2,c3,c4,c5])         

data = {'O0':[22568.2,    23011.9,  23018.7,  23122.7, 22870.3], 
        'O1':[3045.54,    3071.12,  3031.58,  3098.17, 3022.1],
        'O2':[3011.35,    3040.07,  2953.53,  3125.52, 2953.05],
        'O3':[2950.83,    3026.71,  2925.76,  3163.12, 2940.22],
        'Ofast':[2957.44, 2996.12,  3023.72,  3063.46, 2949.75]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.barplot(df,estimator="mean",errorbar=None, palette=cc)

for p in ax.patches:
    ax.annotate(' {:.1f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+100), color='black', size=10)
plt.ylabel("Time (ms)", fontsize=12)
plt.savefig('exp/compiler_opt1.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c1,c2,c3,c4,c5])         

data = {'O0':[8606], 
        'O1':[964],
        'O2':[892],
        'O3':[915],
        'Ofast':[916]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.barplot(df,estimator="mean",errorbar=None, palette=cc)

for p in ax.patches:
    ax.annotate(' {:.0f}'.format(int(p.get_height())), (p.get_x()+0.1, p.get_height()+100), color='black', size=10)
plt.ylabel("Memory (KB)", fontsize=12)
plt.savefig('./exp/compiler_opt_m.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c2,c3,c4,c5])   

data = {#'O0':[22568.2,    23011.9,  23018.7,  23122.7, 22870.3], 
        'O1':[3045.54,    3071.12,  3031.58,  3098.17, 3022.1],
        'O2':[3011.35,    3040.07,  2953.53,  3125.52, 2953.05],
        'O3':[2950.83,    3026.71,  2925.76,  3163.12, 2940.22],
        'Ofast':[2957.44, 2996.12,  3023.72,  3063.46, 2949.75]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

plt.ylim(2800, 3150)
plt.ylabel("Time (ms)", fontsize=12)
sns.barplot(df, errorbar="sd", capsize=0.2, palette=cc)
plt.savefig('exp/compiler_opt2.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c2,c3,c4])   

data = {'mavx mfma':[3005.19,    2980.16,  2984.61, 3000.06, 2980.52], 
        'openmp':[2947,    2914.32,  2939.02,  2917.64, 2934.83],
        'mavx mfma + openmp':[2943.69,    2940.34,  2950.99, 2993.23, 2960]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

plt.ylim(2900, 3020)
plt.ylabel("Time (ms)", fontsize=12)
sns.barplot(df, errorbar="sd", capsize=0.2, palette=cc)
plt.savefig('exp/compiler_opt_openmp.png', dpi=300, bbox_inches='tight')
plt.show()

# 算法复杂度-可视化

In [ ]:
from scripts.visualize import *

proj_name = 'selected'
r = SampleUnion(proj_name)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

x_values = []
y_values = []

for j in sorted(r.results.keys(), key=lambda x: r.results[x].gt_mesh.n_points):
    sample = r.results[j]
    v = sample.gt_mesh.n_points * 1e-3 # -> K 
    t = sample.time * 1e-3 # ms -> s
    x_values.append(v)
    y_values.append(t)

x_data = np.array(x_values)
y_data = np.array(y_values)

def power_function(x, a, b):
    return a * np.power(x, b)

params, covariance = curve_fit(power_function, x_data, y_data)

a_fit, b_fit = params

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)
plt.grid(True)

plt.scatter(x_data, y_data, label='Original data')
x_fit = np.linspace(min(x_data), max(x_data), 100)
plt.plot(x_fit, power_function(x_fit, a_fit, b_fit), color='red', label='Fitted curve')

plt.savefig('exp/compiler_opt2.png', dpi=300, bbox_inches='tight')
plt.xlabel('Vertice nums (K)')
plt.ylabel('Time (s)')
plt.title('Algorithm Time Complexity')

plt.savefig('exp/time_complexity_o2.png', dpi=300, bbox_inches='tight')
plt.show()


print("power_fun a:", a_fit)
print("power_fun b:", b_fit)

# 噪声分析

## Kinect v1

In [ ]:
import igl
import numpy as np
import matplotlib.pyplot as plt

bin_num = 100

def get_sigma(x):
    return np.sqrt(np.mean(np.square(x)))

def get_magnitude(name='big_girl_01', t='Kinect_v1'):
    v1, f = igl.read_triangle_mesh('data/{}/train/noisy/{}_noisy.obj'.format(t, name))
    avg = igl.avg_edge_length(v1, f)
    v2, _ = igl.read_triangle_mesh('data/{}/train/original/{}.obj'.format(t, name))
    v = v2 - v1
    magnitude = np.linalg.norm(v, axis=1)/avg
    return magnitude

plt.figure(figsize=(10, 6))
#plt.title('Kinect v1')

m1 = get_magnitude(name='big_girl_01')
m2 = get_magnitude(name='cone_01')
m3 = get_magnitude(name='david_01')
m4 = get_magnitude(name='pyramid_01')

plt.hist(m1, bins=bin_num, density=True, alpha=0.6, color='b', label='big_girl_01')
plt.hist(m2, bins=bin_num, density=True, alpha=0.6, color='r', label='cone_01')
plt.hist(m3, bins=bin_num, density=True, alpha=0.6, color='g', label='david_01')
plt.hist(m4, bins=bin_num, density=True, alpha=0.6, color='y', label='pyramid_01')

plt.legend()
plt.grid(True)

plt.savefig('exp/noise_analysis_v1.png', dpi=300, bbox_inches='tight')
plt.show()

print('big_girl_01:', get_sigma(m1))
print('cone_01:', get_sigma(m2))
print('david_01:', get_sigma(m3))
print('pyramid_01:', get_sigma(m4))


## Kinect v2

In [ ]:
plt.figure(figsize=(10, 6))
#plt.title('Kinect v2')

m1 = get_magnitude(name='big_girl_01', t='Kinect_v2')
m2 = get_magnitude(name='cone_01', t='Kinect_v2')
m3 = get_magnitude(name='david_01', t='Kinect_v2')
m4 = get_magnitude(name='pyramid_01', t='Kinect_v2')

plt.hist(m1, bins=bin_num, density=True, alpha=0.6, color='b', label='big_girl_01')
plt.hist(m2, bins=bin_num, density=True, alpha=0.6, color='r', label='cone_01')
plt.hist(m3, bins=bin_num, density=True, alpha=0.6, color='g', label='david_01')
plt.hist(m4, bins=bin_num, density=True, alpha=0.6, color='y', label='pyramid_01')

plt.legend()
plt.grid(True)

plt.savefig('exp/noise_analysis_v2.png', dpi=300, bbox_inches='tight')
plt.show()

print('big_girl_01:', get_sigma(m1))
print('cone_01:', get_sigma(m2))
print('david_01:', get_sigma(m3))
print('pyramid_01:', get_sigma(m4))

## Kinect Fusion

In [ ]:
plt.figure(figsize=(10, 6))
#plt.title('Kinect Fusion')

m1 = get_magnitude(name='big_girl', t='Kinect_Fusion')
m2 = get_magnitude(name='cone', t='Kinect_Fusion')
m3 = get_magnitude(name='girl', t='Kinect_Fusion')

plt.hist(m1, bins=bin_num, density=True, alpha=0.6, color='b', label='big_girl')
plt.hist(m2, bins=bin_num, density=True, alpha=0.6, color='r', label='cone_01')
plt.hist(m3, bins=bin_num, density=True, alpha=0.6, color='g', label='girl')

plt.legend()
plt.grid(True)

plt.savefig('exp/noise_analysis_f.png', dpi=300, bbox_inches='tight')
plt.show()

print('big_girl:', get_sigma(m1))
print('cone_01:', get_sigma(m2))
print('girl:', get_sigma(m3))

## Print Data


model_names = ['Ankylosaurus', 
                   'Armor_cat',
                   'Cat_fangs',
                   'Deer',
                   'Direwolf',
                   'Emissary_wolf', 
                   'Girl',
                   'Goblin',
                   'Goku',
                   'Messi',
                   'Minion_ghost',
                   'Nut',
                   'Putin',
                   'Snoopy',
                   'Snoopy_flying_face',
                   'Sphagetti_Detective',
                   'Stitch_guitar',
                   'Stitch_stand',
                   'Suit_man',
                   'Tp']



In [ ]:
import math
import numpy as np
import open3d as o3d

from tqdm import tqdm
from sklearn import neighbors
def compute_vert_distance_error_with_pc_kdtree(scanned_mesh, gt_pc):
    ret = []
    scanned_vertices = np.asarray(scanned_mesh.vertices)
    gt_centroids = np.asarray(gt_pc.points)
    kd_tree = neighbors.KDTree(gt_centroids)
    for i in tqdm(range(scanned_vertices.shape[0])):
        query_point = [scanned_vertices[i]]
        dist, ind = kd_tree.query(query_point, k = 1)
        ret.append(dist[0])

    return np.array(ret)

def get_magnitude(name='Tp'):
    gt_pc = o3d.io.read_point_cloud('data/PrintedDataset/gt_pc/{}.ply'.format(name))
    mesh = o3d.io.read_triangle_mesh('data/PrintedDataset/noisy/{}.obj'.format(name))
    ret = compute_vert_distance_error_with_pc_kdtree(mesh, gt_pc)
    ret = ret.squeeze()

    v1, f = igl.read_triangle_mesh('data/PrintedDataset/noisy/{}.obj'.format(name))
    avg = igl.avg_edge_length(v1, f)
    ret = (1/avg) * ret
    return ret

plt.figure(figsize=(10, 6))
#plt.title('Printed Data')

m1 = get_magnitude(name='Tp')
m2 = get_magnitude(name='Nut')
m3 = get_magnitude(name='Goblin')

plt.hist(m1, bins=bin_num, density=True, alpha=0.6, color='b', label='Tp')
plt.hist(m2, bins=bin_num, density=True, alpha=0.6, color='r', label='Nut')
plt.hist(m3, bins=bin_num, density=True, alpha=0.6, color='g', label='Goblin')

plt.legend()
plt.grid(True)

plt.savefig('exp/noise_analysis_p.png', dpi=300, bbox_inches='tight')
plt.show()

print('Tp:', np.std(m1))
print('Nut:', np.std(m2))
print('Goblin:', np.std(m3))